# Tarea 2: Naive Bayes, Linear Models y Neural Networks
**Procesamiento de Lenguaje Natural (CC6205-1 - Otoño 2024)**

## Tarjeta de identificación

**Nombres:** ```Sebastián Sanhueza y Martín Reyes Oviedo```

**Fecha límite de entrega 📆:** 30/04.

**Tiempo estimado de dedicación:** 4 horas


## Instrucciones
Bienvenid@s a la segunda tarea en el curso de Natural Language Processing (NLP). Esta tarea tiene como objetivo evaluar los contenidos teóricos de las últimas semanas de clases posteriores a la tarea 1, enfocado principalmente en **Naive Bayes**, **Linear Models** y **Neural Networks**. Si aún no has visto las clases, se recomienda visitar los links de las referencias.

La tarea consta de una una parte práctica con el fín de introducirlos a la programación en Python enfocada en NLP.

* La tarea es en **grupo** (maximo hasta 3 personas).
* La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
* El formato de entrega es este mismo Jupyter Notebook.
* Al momento de la revisión su código será ejecutado. Por favor verifiquen que su entrega no tenga errores de compilación.
* Completar la tarjeta de identificación. Sin ella no podrá tener nota.

> **Importante:** Esta tarea tiene varios resultados experimentales que pueden variar de acuerdo a sus propias implementaciones. No se busca que los resultados sean exactamente los mismos (por ejemplo, que el accuracy fue el mismo que el que esta en la tarea). Lo importante es que implementen sus funciones, las sepan explicar y que puedan hacer varios experimentos.

## Material de referencia

Diapositivas del curso 📄
    
- [Naive Bayes](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-NB.pdf)
- [Linear Models](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-linear.pdf)
- [Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-neural.pdf)

Videos del curso 📺

- Naive Bayes: [Parte 1](https://www.youtube.com/watch?v=kG9BK9Oy1hU), [Parte 2](https://www.youtube.com/watch?v=Iqte5kKHvzE), [Parte 3](https://www.youtube.com/watch?v=TSJg0_X3Abk)

- Linear Models: [Parte 1](https://www.youtube.com/watch?v=zhBxDsNLZEA), [Parte 2](https://www.youtube.com/watch?v=Fooua_uaWSE), [Parte 3](https://www.youtube.com/watch?v=DqbzhdQa1eQ), [Parte 4](https://www.youtube.com/watch?v=1nfWWXqfAzA)

- Neural Networks: [Parte 1](https://www.youtube.com/watch?v=oHZHA8h2xN0), [Parte 2](https://www.youtube.com/watch?v=2lXank0W6G4), [Parte 3](https://www.youtube.com/watch?v=BUDIi9qItzY), [Parte 4](https://www.youtube.com/watch?v=KKN2Ipy-vGk)


## P0. Cargar un dataset

Importamos algunas librerias que seran utiles.

In [1]:
import pandas as pd
from collections import namedtuple

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Inicializamos el dataset con particiones de entrenamiento y test. Es un dataset de clasificacion multi-clase de oraciones. Cada oracion puede tener una unica etiqueta ?, + o -. Donde ? indica que la oracion es una pregunta, - que la oracion es negativa y + positiva.

In [2]:
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

raw_train_set = [
              ['Do you have plenty of time?', '?'],
              ['Does she have enough money?','?'],
              ['Did they have any useful advice?','?'],
              ['What day is today?','?'],
              ["I don't have much time",'-'],
              ["She doesn't have any money",'-'],
              ["They didn't have any advice to offer",'-'],
              ['Have you plenty of time?','?'],
              ['Has she enough money?','?'],
              ['Had they any useful advice?','?'],
              ["I haven't much time",'-'],
              ["She hasn't any money",'-'],
              ["He hadn't any advice to offer",'-'],
              ['How are you?','?'],
              ['How do you make questions in English?','?'],
              ['How long have you lived here?','?'],
              ['How often do you go to the cinema?','?'],
              ['How much is this dress?','?'],
              ['How old are you?','?'],
              ['How many people came to the meeting?','?'],
              ['I’m from France','+'],
              ['I come from the UK','+'],
              ['My phone number is 61709832145','+'],
              ['I work as a tour guide for a local tour company','+'],
              ['I’m not dating anyone','-'],
              ['I live with my wife and children','+'],
              ['I often do morning exercises at 6am','+'],
              ['I run everyday','+'],
              ['She walks very slowly','+'],
              ['They eat a lot of meat daily','+'],
              ['We were in France that day', '+'],
              ['He speaks very fast', '+'],
              ['They told us they came back early', '+'],
              ["I told her I'll be there", '+']
]
tokenized_train_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_train_set]
train_set = pd.DataFrame(data=tokenized_train_set)

raw_test_set = [
             ['Do you know who lives here?','?'],
             ['What time is it?','?'],
             ['Can you tell me where she comes from?','?'],
             ['How are you?','?'],
             ['I fill good today', '+'],
             ['There is a lot of history here','+'],
             ['I love programming','+'],
             ['He told us not to make so much noise','+'],
             ['We were asked not to park in front of the house','+'],
             ["I don't have much time",'-'],
             ["She doesn't have any money",'-'],
             ["They didn't have any advice to offer",'-'],
             ['I am not really sure','+']
]
tokenized_test_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_test_set]
test_set = pd.DataFrame(data=tokenized_test_set)

Separar en X e y, donde X son oraciones tokenizadas e y es la clase a predecir (o target).

In [3]:
X_train, y_train = train_set.drop(columns="class_"), train_set["class_"]
pd.concat([X_train, y_train], axis=1).sample(10)

,words,class_
11,"(she, has, n't, any, money)",-
33,"(i, told, her, i, 'll, be, there)",+
30,"(we, were, in, france, that, day)",+
6,"(they, did, n't, have, any, advice, to, offer)",-
18,"(how, old, are, you, ?)",?
19,"(how, many, people, came, to, the, meeting, ?)",?
0,"(do, you, have, plenty, of, time, ?)",?
29,"(they, eat, a, lot, of, meat, daily)",+
24,"(i, ’, m, not, dating, anyone)",-
28,"(she, walks, very, slowly)",+


Cantidad de oraciones por clase:

In [4]:
train_set.groupby("class_").count()

,words
class_,
+,13
-,7
?,14


(X, y) para el conjunto de test:

In [5]:
X_test, y_test = test_set.drop(columns="class_"), test_set["class_"]
pd.concat([X_test, y_test], axis=1).sample(10)

,words,class_
7,"(he, told, us, not, to, make, so, much, noise)",+
3,"(how, are, you, ?)",?
6,"(i, love, programming)",+
11,"(they, did, n't, have, any, advice, to, offer)",-
8,"(we, were, asked, not, to, park, in, front, of...",+
5,"(there, is, a, lot, of, history, here)",+
0,"(do, you, know, who, lives, here, ?)",?
1,"(what, time, is, it, ?)",?
10,"(she, does, n't, have, any, money)",-
4,"(i, fill, good, today)",+


Cantidad de oraciones por clase en el conjunto de test:

In [6]:
test_set.groupby("class_").count()

,words
class_,
+,6
-,3
?,4


**Importante:** Hasta el momento hemos creado nuestros conjuntos de train y test. A continuacion ustedes deben implementar tres modelos de clasificacion: Naive-bayes, Linear Model y Neural Network. Aqui va un resumen de cada pregunta y lo que se les pide implementar:

* P1: Naive-bayes
 - Implementar `fit` y `predict`
 - Entrenar
 - Evaluar

* P2: Linear Model
 - Implementar `fit` con *on-line gradient descent* y `predict`
 - Entrenar
 - Evaluar

* P3: Neural Network
 - Implementar un iterador de datos con `datasets` y `dataloaders`
 - Implementar una red neuronal con `pytorch`
 - Implementar loop de entrenamiento de una NN
 - Entrenar
 - Evaluar

## P1. Implementar y evaluar Multinomial Naive-Bayes (2 puntos)

### Clase para clasificador

Cree una clase MyMultinomialNB que en su inicializador reciba el parámetro alpha para su clasficador.

Además, debe implementar los métodos `fit(X, y)`y `predict(X)`.

```python
class MyMultinomialNB():
  def __init__(self, alpha, ...):
    ...

  def fit(self, X, y):
    ...
  
  def predict(self, X):
    ...
    return prediction
```
Para computar el entrenamiento de nuestro clasificador debemos:
- extraer el vocabulario,
- determinar las probabilidades $p(c_j)$ para cada una de las clases posibles,
- determinar las probabilidades $p(w_i|c_j)$ para cada una de las palabras y cada una de las clases.

Para lograr lo anterior, también deberán implementar el método `predict_proba(X)`:

```python
  def predict_proba(self, X):
    return prob
```

**Underflow prevention:** En vez de hacer muchas multiplicaciones de `float`s, reemplácenlas por sumas de logaritmos para prevenir errores de precisión. (Revisen la diapo 26 de las slides).

En su implementación deben considerar la tecnica de *Laplace Smoothing* vista en clases. Especificamente considere que su clase `MyMultinomialNB` reciba un parámetro `alpha` no negativo (es decir, mayor o igual a cero). De tal forma que el la probabilidad de una palabra $w$ dado la clase $c$ viene dado por lo siguiente:

$$
p_\alpha (w|c) = \frac{\#(w, c) + \alpha}{N + \alpha |V|}
$$

donde $\alpha$ es el parámetro `alpha` de *Laplace Smoothing*. Mientras que los otras notaciones corresponden a

* $\#(w, c)$ numero de veces que ocurre la palabra $w$ en documentos con la clase $c$ (pensar en un gran documento $D_c$ que concatena todos los documentos de clase $c$ y luego calcula la frecuencia de la palabra $w$ en $D_c$),
* $N$ es igual a $\sum \{\#(w', c): w' \in V\}$ donde $V$ es el vocabulario,
* $|V|$ tamaño del vocabulario.

### Implementación (1.5 pts.)

Escriba aquí la implementación de la clase `MyMultinomialNB`.

In [32]:
import numpy as np

class MyMultinomialNB():
  def __init__(self, alpha=1.0):
    self.alpha = alpha

  def fit(self, X, y):
    """Ajusta el modelo a partir de datos de entrenamiento

    Args:
      X: Serie de pandas con documentos
      y: Serie de pandas con clases ("class_") de los documentos

    Returns:
      None
    """
    corpus = []
    train = np.concatenate(X.to_numpy())
    for tupla in train:
      for j in tupla:
        corpus.append(j)
    vocab = {}
    for word in corpus:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1
    vocab_per_class = {}
    for clas in list(set(y.to_numpy())):
      corpus_c = []
      train_c = np.concatenate(X[y == clas].to_numpy())
      for tupla in train_c:
        for j in tupla:
          corpus_c.append(j)
      vocabpc = {}
      for word in corpus_c:
        if word in vocabpc:
          vocabpc[word] += 1
        else:
          vocabpc[word] = 1
      vocab_per_class[clas] = vocabpc
    self.clases = list(set(y.to_numpy()))
    self.priors = [sum(y_train == c)/len(y_train) for c in self.clases]
    self.vocab_per_class = vocab_per_class
    self.vocab = vocab
    self.v = len(self.vocab)
    self.n = [np.sum(list(self.vocab_per_class[i].values())) for i in self.clases]




  def predict(self, X):
    """Predice las clases más probables de una serie de documentos

    Args:
      X: Serie de pandas con documentos

    Returns:
      Serie de pandas con las clase de cada documento de X
    """
    limpio = np.concatenate(X.to_numpy())
    probs = np.zeros((len(limpio), len(self.clases)))
    for i in range(len(limpio)):
      for j in range(len(self.clases)):
        probabilidades = []
        for word in limpio[i]:
          try:
            prob = (self.vocab_per_class[self.clases[j]][word] + self.alpha) / (self.n[j] + self.alpha * self.v)
          except KeyError:  # Manejar la excepción si la palabra no está en el vocabulario
            prob = self.alpha / (self.n[j] + self.alpha * self.v)
          probabilidades.append(prob)
        probs[i,j] = np.prod(probabilidades)*self.priors[j]
        #probs[i,j] = [try: ((self.vocab_per_class[self.clases[j]][word]+ self.alpha)/(self.n[j]+ self.alpha*self.v)) except: ((self.alpha)/(self.n[j]+ self.alpha*self.v))  for word in limpio[i]]#*self.priors[j]
    argmax_c = np.argmax(probs, axis=1)
    pred = [self.clases[idx] for idx in argmax_c]
    return pred

### Entrenamiento (0.2 pts.)
A continuación, inicialicen y entrenen (ajusten) su clasificador con los datos de entrenamiento.

In [33]:
nb_model = MyMultinomialNB(alpha=0.1)
nb_model.fit(X_train, y_train);

Pruébenlo utilizando el método `predict()` que implementaron.

In [18]:
from sklearn.metrics import classification_report

In [35]:
# Predict train-set
y_pred = nb_model.predict(X_train)
print(y_pred)

['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']


In [36]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       1.00      1.00      1.00        13
           -       1.00      1.00      1.00         7
           ?       1.00      1.00      1.00        14

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.


In [37]:
y_pred_test = nb_model.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           +       1.00      0.50      0.67         6
           -       0.50      1.00      0.67         3
           ?       1.00      1.00      1.00         4

    accuracy                           0.77        13
   macro avg       0.83      0.83      0.78        13
weighted avg       0.88      0.77      0.77        13



Comenten sus resultados. Estudien que ocurre para alpha=0, 1 y L donde L es un numero muy grande.

**Comentarios:**

Se observa que en términos de rendimiento en la clasificación del modelo, la clase '?' fue la más fácil de identificar, posiblemente debido a que todas las oraciones de esta clase contienen el token '?'. Sin embargo, hubo mayores dificultades para distinguir entre las clases '+' y '-', lo que afectó tanto al recall como a la precisión de estas respectivas clases.

En cuanto a los valores de alpha = 0.1 y un vocabulario de tamaño muy grande, se nota que todas las probabilidades se vuelven extremadamente pequeñas, lo que lleva a problemas cuando se multiplican las probabilidades y estas tienden a acercarse a cero. Esta situación se resuelve comúnmente aplicando el logaritmo a las probabilidades, lo que facilita su manipulación y evita la pérdida de precisión debido a la multiplicación de valores muy pequeños.


## P2. Implementar y evaluar Linear Models (2 puntos)

### Clase para clasificador

Cree una clase MyLinearModel para su clasficador. Debe implementar los métodos `fit(X, y, learning_rate, epochs)`y `predict(X)`.

```python
class MyLinearModel():
  def __init__(self, ...):
    ...

  def fit(self, X, y, learning_rate, epochs):
    ...
  
  def predict(self, X):
    ...
    return prediction
```

El modelo lineal que debe implementar viene dado por:
$$
\vec{\hat{y}} = \text{softmax}(\vec{x} \cdot W + \vec{b})\\
\vec{\hat{y}}_{[i]} = \frac{\exp{z_i}}{\sum_{j} \exp{z_j}}\\
z_i = \vec{x} \cdot W_{[:, i]} + \vec{b}_{[i]}
$$
donde $\vec{x}$ es un documento representado con bolsas de palabras (BoW), $W$ es la matriz de pesos y $\vec{b}$ el bias.

El modelo linea debe ajustarlo considerando como objetivo minimizar la cross-entropy loss, es decir:

$$
L_\text{cross-entropy}(\vec{\hat{y}}, \vec{y}) = - \sum_i \vec{y}_{[i]} \log{ \left( \vec{\hat{y}}_{[i]} \right) }
$$

Para representar un documento `(i, am, not, really, sure)` vectorialmente, utilice `CountVectorizer` de sklearn. De esta manera, el documento queda representado como sigue:

|    |   i |   he |   am |   are |   not |   yes |   really |   sure |
|---:|----:|-----:|-----:|------:|------:|------:|---------:|-------:|
|  0 |   1 |    0 |    1 |     0 |     1 |     0 |        1 |      1 |

**Observación:** Si el documento repite palabras entonces tendrá un número mayor a 1. Si el documento no tiene la palabra entonces tiene un 0. Pensar que las palabras `(he, are, yes)` provienen de otros documentos. Recuerde que el `CountVectorizer` se entrena con más de un documento (es decir, un corpus). Aquí debe usar el conjunto de train.

El método `fit(X, y, learning_rate, epochs)` debe ajustar un `CountVectorizer` para representar vectorialmente el documento. Debe guardar el `CountVectorizer` para cuando quiera hacer predicciones. Dentro del método `fit(X, y, learning_rate, epochs)` debe implementar *On-line gradient descent* (visto en clases), es decir, descenso de gradiente usando un data-point por iteración. Su método debe ser capaz de recibir un `learning_rate` para ponderar el gradiente en cada iteración y fijar un número de `epochs`. Luego de entrenar debe guardar los pesos de su modelo lineal, es decir, $(W, \vec{b})$.

En el algoritmo de descenso de gradiente usando un data-point por iteración, o *On-line gradient descent*, debe implementar manualmente las derivadas. Como conoce el modelo lineal y la funcion objetivo, entonces puede calcular manualmente las derivadas. Para ejemplificar, en cada paso del algoritmo de optimizacion debe actualizar los pesos $(W, \vec{b})$ del siguiente modo:

$$
W \leftarrow W - \lambda \nabla_{W} L_\text{cross-entropy}\\
\vec{b} \leftarrow \vec{b} - \lambda \nabla_{\vec{b}} L_\text{cross-entropy}\\
$$
donde $\lambda$ es el parámetro `learning_rate`, $\nabla_{W} L_\text{cross-entropy}$ el gradiente de la Loss con repecto a la matriz de pesos $W$ y $\nabla_{\vec{b}} L_\text{cross-entropy}$ para el bias $\vec{b}$.

Para implementar el algoritmo *On-line gradient descent* les recomendamos (no es obligatorio hacerlo de este modo) definir una función `get_derivative_W(x, y_target, y_pred, n_classes)` que calcule $\nabla_{W} L_\text{cross-entropy}$ y lo mismo con una función `get_derivative_b(y_target, y_pred, n_classes)` que calcule $\nabla_{\vec{b}} L_\text{cross-entropy}$.

Para implementar el método `predict(self, X)` debera usar su `CountVectorizer` definido en `fit(X, y, learning_rate, epochs)` para representar del mismo modo cualquier documento tanto en train como en test.

### Implementación (1.5 pts.)
Implemente un modelo lineal con métodos `fit(X, y)` y `predict(X)`

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def softmax(x):
    """Función para calcular la función softmax de un arreglo de números.

    Args:
      x : ndarray
          Arreglo de entrada.

    Returns:
      ndarray : Arreglo con la función softmax aplicada a cada fila del arreglo de entrada.

    """
    # Resta el máximo de cada fila para evitar el overflow numérico
    exp_x = np.exp(x - np.max(x, keepdims=True))
    # Calcula la suma de las exponenciales
    return exp_x / np.sum(exp_x, keepdims=True)




def get_derivative_W(x, y_target, y_pred, n_classes):
    """Calcula la derivada de la función de pérdida respecto a la matriz de pesos W.

    Args:
      x : ndarray
          Vector de características.
      y_target : ndarray
          Vector de clases objetivo codificado en one-hot.
      y_pred : ndarray
          Vector de probabilidades predichas por el modelo.
     n_classes : int
          Número de clases.

    Returns:
      ndarray : Matriz de derivadas de la función de pérdida respecto a W.

    """
    return np.outer(x, (y_pred - y_target))




def get_derivative_b(y_target, y_pred, n_classes):
    """Calcula la derivada de la función de pérdida respecto al vector de sesgos b.

    Args:
      y_target : ndarray
          Vector de clases objetivo codificado en one-hot.
      y_pred : ndarray
          Vector de probabilidades predichas por el modelo.
      n_classes : int
          Número de clases.

    Returns:
      ndarray : Vector de derivadas de la función de pérdida respecto a b.

    """
    return y_pred - y_target




def get_preds_tests(X, y, linear_layer):
    """Obtiene las predicciones y las clases reales para un conjunto de datos.

    Args:
      X : list
          Lista de documentos.
      y : ndarray
          Vector de clases reales.
      linear_layer : MyLinearModel
          Instancia del modelo lineal.

    Returns:
      tuple : Tupla que contiene el vector de predicciones y el vector de clases reales.

    """
    # Transforma los documentos en una representación de texto
    X_text = X.words.apply(lambda x: ' '.join(x)).values
    # Transforma los documentos a su representación vectorial
    X_vec = linear_layer.vectorizer.transform(X_text).toarray()
    # Calcula las puntuaciones para cada clase
    z = np.dot(X_vec, linear_layer.W) + linear_layer.b
    # Aplica la función softmax para obtener las probabilidades
    y_pred = softmax(z)
    # Devuelve las clases predichas y las reales
    return y_pred.argmax(axis=1), y




class MyLinearModel():

  def __init__(self):
      """Inicializa el modelo

      Returns:
      None

      """
      self.vectorizer = None # Objeto CountVectorizer para convertir texto a vectores.
      self.W = None # Matriz de pesos del modelo lineal.
      self.b = None # Vector de sesgos del modelo lineal.
      self.class_mapping = None # Mapeo de símbolos de clase a valores numéricos únicos.
      self.reverse_class_mapping = None # Mapeo inverso de valores numéricos de clase a símbolos de clase.

  def fit(self, X, y, learning_rate, epochs, verbose=False):
      """Entrena el modelo a partir de datos de entrenamiento

      Args:
        X: Serie de pandas con documentos
        y: Serie de pandas con clases ("class_") de los documentos
        learning_rate: parametro learning rate del modelo
        verbose: para imprimir mensajes de progreso

      Returns:
        None

      """
      # Mapea los símbolos de clase a valores numéricos únicos
      self.class_mapping = {symbol: i for i, symbol in enumerate(np.unique(y))}
      # Crea el mapeo inverso
      self.reverse_class_mapping = {i: symbol for symbol, i in self.class_mapping.items()}

      # Transforma los documentos en texto
      X_text = X.words.apply(lambda x: ' '.join(x)).values
      # Inicializa el vectorizador y convierte los documentos a representaciones vectoriales
      self.vectorizer = CountVectorizer()
      X_vec = self.vectorizer.fit_transform(X_text).toarray()

      n_classes = len(self.class_mapping)
      n_features = X_vec.shape[1]
      # Inicializa los pesos y sesgos del modelo como matrices de ceros
      self.W = np.zeros((n_features, n_classes))
      self.b = np.zeros(n_classes)

      # Entrenamiento del modelo durante el número de épocas especificado
      for epoch in range(epochs):
        total_loss = 0.0
        # Itera sobre cada ejemplo del conjunto de entrenamiento
        for i, x_i in enumerate(X_vec):
          y_i = np.zeros(n_classes)
          y_i[self.class_mapping[y[i]]] = 1 # Codificación one-hot de la clase objetivo del ejemplo actual

          z = np.dot(x_i, self.W) + self.b # Calcula las puntuaciones
          y_pred = softmax(z) # Calcula las probabilidades predichas

          # Calcula la cross entropy loss
          loss = -np.sum(y_i * np.log(y_pred))
          total_loss += loss

          # Calcula los gradientes de los pesos y sesgos
          dW = get_derivative_W(x_i, y_i, y_pred, n_classes)
          db = get_derivative_b(y_i, y_pred, n_classes)
          # Actualiza los pesos y sesgos utilizando el descenso de gradiente
          self.W -= learning_rate * dW
          self.b -= learning_rate * db

        # Calcula la pérdida promedio en esta época
        avg_loss = total_loss / len(X_vec)
        if verbose is True:
          print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

  def predict(self, X):
      """Predice las clases más probables de una serie de documentos

      Args:
        X: Serie de pandas con documentos

      Returns:
        Serie de pandas con las clase de cada documento de X

      """
      # Transforma los documentos en texto
      X_text = X.words.apply(lambda x: ' '.join(x)).values
      # Convierte los documentos a su representación vectorial
      X_vec = self.vectorizer.transform(X_text).toarray()
      # Calcula las puntuaciones para cada clase
      z = np.dot(X_vec, self.W) + self.b
      # Calcula las probabilidades predichas usando la función softmax
      y_pred = softmax(z)
      # Devuelve las clases predichas como el símbolo de clase correspondiente
      return [self.reverse_class_mapping[np.argmax(pred)] for pred in y_pred]

### Entrenamiento (0.2 pts.)
Inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [19]:
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.02,
    epochs=15,
    verbose=True)

Epoch 1/15, Loss: 1.0774
Epoch 2/15, Loss: 0.9725
Epoch 3/15, Loss: 0.8875
Epoch 4/15, Loss: 0.8167
Epoch 5/15, Loss: 0.7566
Epoch 6/15, Loss: 0.7047
Epoch 7/15, Loss: 0.6595
Epoch 8/15, Loss: 0.6196
Epoch 9/15, Loss: 0.5843
Epoch 10/15, Loss: 0.5526
Epoch 11/15, Loss: 0.5242
Epoch 12/15, Loss: 0.4984
Epoch 13/15, Loss: 0.4750
Epoch 14/15, Loss: 0.4535
Epoch 15/15, Loss: 0.4339


Pruébenlo utilizando el método `predict()` que implementaron.

In [20]:
from sklearn.metrics import classification_report

In [21]:
# Predict train-set
y_pred = linear_model.predict(X_train)
print(f'\nClases reales: {list(y_train)}')
print(f'\nClases predichas: {y_pred}')


Clases reales: ['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']

Clases predichas: ['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']


In [22]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       1.00      1.00      1.00        13
           -       1.00      1.00      1.00         7
           ?       1.00      1.00      1.00        14

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.

In [23]:
y_pred = linear_model.predict(X_test)
print(f'\nClases reales: {list(y_test)}')
print(f'\nClases predichas: {y_pred}')
print('\n')
print(classification_report(y_test, y_pred))


Clases reales: ['?', '?', '?', '?', '+', '+', '+', '+', '+', '-', '-', '-', '+']

Clases predichas: ['?', '?', '?', '?', '+', '+', '+', '+', '+', '-', '-', '-', '+']


              precision    recall  f1-score   support

           +       1.00      1.00      1.00         6
           -       1.00      1.00      1.00         3
           ?       1.00      1.00      1.00         4

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de learning rates y epochs, por ejemplo `learning_rate, epochs = (0.02, 15), (0.1, 10), (0.005, 30)`.

**Comentarios:**

Los resultados muestran que el modelo, tras el entrenamiento, logró una excelente generalización de los datos. Esto se evidencia en los valores de accuracy, precision, recall, y f1-score, los cuales alcanzaron un puntaje perfecto de 1.0. Estos resultados indican que el modelo clasificó correctamente todas las frases del conjunto de prueba. Aunque en el proceso de entrenamiento también se obtuvieron los valores máximos para cada métrica, lo cual podría sugerir overfitting, este fenómeno no se manifestó durante la evaluación con el conjunto de prueba.

A continuación se procederá a estudiar los resultados obtenidos con distintas combinaciones de learning rate y épocas.


In [24]:
def evaluate_learning_rates_epochs(modelos, X_train, y_train, X_test, y_test):
    """Evalúa diferentes combinaciones de learning rates y épocas para un modelo lineal.

    Args:
      modelos: ndarray de tuplas
          Cada tupla contiene un learning rate y un número de épocas.
      X_train: Serie de pandas
            Conjunto de datos de entrenamiento.
      y_train: Serie de pandas
            Etiquetas de clase para el conjunto de entrenamiento.
      X_test: Serie de pandas
            Conjunto de datos de prueba.
      y_test: Serie de pandas
            Etiquetas de clase para el conjunto de prueba.

    Returns:
      None

    """
    for lr, epochs in modelos:

        print(f"Evaluando modelo con learning rate={lr} y epochs={epochs}")

        # Inicializa el nuevo modelo lineal
        linear_model = MyLinearModel()
        # Ajusta el modelo lineal con el conjunto de entrenamiento
        linear_model.fit(X_train, y_train, learning_rate=lr, epochs=epochs, verbose=True)
        # Predice las clases para el conjunto de entrenamiento
        y_pred_train = linear_model.predict(X_train)

        print("\nPredicciones para el conjunto de entrenamiento:")
        print(f'Clases reales: {list(y_train)}')
        print(f'Clases predichas: {y_pred_train}')

        # Predice las clases para el conjunto de prueba
        y_pred_test = linear_model.predict(X_test)

        print("\nPredicciones para el conjunto de prueba:")
        print(f'Clases reales: {list(y_test)}')
        print(f'Clases predichas: {y_pred_test}')

        # Imprime el informe de clasificación para el conjunto de prueba
        print('\nInforme de clasificación para el conjunto de prueba:')
        print(classification_report(y_test, y_pred_test))

        print('\n' + '='*50 + '\n')

Para evaluar mejor el modelo a continuación se estudiarán los resultados obtenidos para distintas combinaciones de learning rate y épocas. En particular se evaluará para: (0.01, 10), (0.02, 15), (0.05, 20). Donde el primer valor corresponde al learning rate y el segundo a la épocas.

In [25]:
# Valores de learning rate y épocas a evaluar
modelos = [(0.01, 10), (0.02, 15), (0.05, 20)]

# Evaluación del modelo lineal con los valores especificados
evaluate_learning_rates_epochs(modelos, X_train, y_train, X_test, y_test)

Evaluando modelo con learning rate=0.01 y epochs=10
Epoch 1/10, Loss: 1.0878
Epoch 2/10, Loss: 1.0307
Epoch 3/10, Loss: 0.9799
Epoch 4/10, Loss: 0.9344
Epoch 5/10, Loss: 0.8932
Epoch 6/10, Loss: 0.8558
Epoch 7/10, Loss: 0.8214
Epoch 8/10, Loss: 0.7898
Epoch 9/10, Loss: 0.7606
Epoch 10/10, Loss: 0.7335

Predicciones para el conjunto de entrenamiento:
Clases reales: ['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']
Clases predichas: ['?', '?', '?', '?', '?', '-', '-', '?', '?', '?', '?', '?', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+']

Predicciones para el conjunto de prueba:
Clases reales: ['?', '?', '?', '?', '+', '+', '+', '+', '+', '-', '-', '-', '+']
Clases predichas: ['?', '?', '?', '?', '+', '+', '+', '+', '+', '?', '-', '-', '+']

Informe de clasificación para el conjunto de prueba:
            

**Comentarios:**

Los resultados anteriores se analizaron con el objetivo de entender cómo influye el aumento del learning rate y el número de épocas de entrenamiento en el desempeño del modelo. Se observó que incluso con valores bajos, como un learning rate de 0.01 y 10 épocas, se obtuvieron resultados satisfactorios, con altos niveles de precision y recall en la mayoría de las clases. Específicamente, para las frases positivas, todas fueron clasificadas correctamente, mientras que para las negativas, a pesar de una precision del 100% (sin falsos positivos), el recall fue del 67%, indicando una proporción desfavorable entre verdaderos positivos y falsos negativos, lo que también se reflejó en el f1-score. En cuanto a las frases de consulta, se logró un 100% de recall pero solo un 80% de precision, lo que sugiere una proporción problemática entre verdaderos positivos y falsos positivos. Sin embargo, el accuracy general fue del 92%, respaldado por un f1-score promedio ponderado del 92%, indicando un equilibrio satisfactorio entre precision y recall en todas las clases evaluadas.

Utilizando un learning rate de 0.02 y 15 épocas de entrenamiento se obtuvieron los mejores resultados. Se logró un precision y recall perfectos del 100% para todas las clases: positiva, negativa y de tipo consulta. Esto indica que todas las instancias fueron clasificadas correctamente, sin ningún tipo de error en ninguna de las clases. Este nivel de desempeño se refleja en un accuracy del 100%, respaldado por un f1-score perfecto del 100% para todas las clases. Estos resultados sugieren que el modelo pudo capturar de manera precisa y completa las características distintivas de cada clase, demostrando una capacidad de generalización muy buena en la clasificación de nuevas instancias.


Los resultados obtenidos para el conjunto de prueba utilizando un learning rate de 0.05 y 20 épocas de entrenamiento muestran un buen rendimiento, aunque con algunas diferencias respecto a los experimentos anteriores. Se observa un precision del 100% para las clases positiva y de tipo consulta, lo que indica que todas las instancias clasificadas como tales fueron correctas. Sin embargo, para la clase negativa, aunque se alcanzó un recall perfecto del 100%, el precision fue del 75%, lo que sugiere una proporción desigual entre los verdaderos positivos y los falsos positivos. Esto se traduce en un f1-score más bajo para esta clase en comparación con las otras dos. A pesar de estas variaciones, el modelo mantuvo un accuracy general del 92%, respaldado por un f1-score promedio ponderado del 93%, lo que sugiere un equilibrio satisfactorio entre precision y recall en todas las clases evaluadas, aunque con un ligero sesgo hacia las frases positivas y de tipo consulta.

En conclusión, los tres experimentos mostraron resultados significativos en la clasificación de las frases. El primero tuvo un buen rendimiento general, aunque con ciertas limitaciones en frases negativas. El segundo experimento alcanzó una clasificación perfecta en todas las clases. En el tercero, se mantuvo un buen rendimiento, aunque con una ligera disminución en presicion para las frases negativas. Estos resultados resaltan la importancia de ajustar los hiperparámetros para lograr un equilibrio óptimo en la clasificación, ya que no necesariamente por aumentar las épocas o el valor del learning rate se obtendrán mejores resultados.

## P3. Implementar y evaluar Neural Networks (2 puntos)

### Especificaciones del clasificador

<img src="https://docs.google.com/drawings/d/e/2PACX-1vSXJm5I61m6w0RHTwBL-iMyeFLr2wXBrKNYxdU8Bu1ymuCFPD9dAPsCzPfvIqwSr8uCiYvWMdnGy1if/pub?w=818&h=503" >

En esta última pregunta, ustedes deberánimplementar y evaluar redes neuronales (como la de la figura de arriba). Para esto debera implementar tres secciones principales:

1. Sección iterador,
2. Sección modelo, y
3. Sección loop de entrenamiento.

> **Recomendación:** Para completar esta pregunta puede guiarse del Auxiliar 2 (clase del día 18/04).

*Seccion iterador*

Para ayudarnos a con el entrenamiento y testing, vamos a utilizar las clases `Dataset` y `DataLoader` de `pytorch` ([ver documentación](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)). En esta sección deberá implementar un contenedor para su conjunto de datos usando la clase `Dataset` de `pytorch`. Para esto deberá crear su propia clase `MyDataset` para gestionar los datos. Ésto le permitirá iterar sobre el conjunto mediante el iterador `DataLoader` de `pytorch` y entrenar sin hacer ningún pre-procesamiento extra a los datos.

**Observación:** Si considera por funcionalidad cambiar los parámetros de la clase `MyDataset` puede hacerlo. Asimismo, puede definir otros parámetros para los métodos de su clase.


```python
class MyDataset(Dataset):
    def __init__(self, data, bow_cols):
      ...

    def __len__(self):
      ...

    def __getitem__(self, index):
      ...
      return x_bow, label
```

*Sección modelo*

En esta sección deberán implementar la clase `MyNeuralNetwork` del modulo de `pytorch` llamado `nn.Module` con el proposito de diseñar una red neuronal como la figura de arriba. Para mas detalle sobre las redes ver Clase NLP-Neural.pdf Slide número 8.

**Observación:** La figura de arriba es solo ilustrativa, ustedes pueden variar la dimension input y output de la capa oculta. Sin embargo deben mantener fija la dimension de la entrada y salida de la red. La entrada depende del tamaño del vocabulario. Mientras que la salida depende de la cantidad de clases de su problema de clasificación (en nuestro caso igual a 3).

Es importante que la clase `MyNeuralNetwork` tenga implementadas apropiadamente el `__init__` con las dimensiones y el `forward` con entrada tipo BoW retornando el último estado de la red (output layer). En el `forward` recomendamos utilizar funciones de activación tipo `nn.ReLU`. Sin embargo, no es completamente obligatorio por lo que pueden usar otras.

```python
class MyNeuralNetwork(nn.Module):
    def __init__(self,
                 dim_vocab,
                 num_classes,
                 dim_hidden_input,
                 dim_hidden_output):

        super(MyNeuralNetwork, self).__init__()
        torch.manual_seed(42)
      ...

    def forward(self, xs_bow):
      ...
      return last_state
  ```

*Sección loop de entrenamiento*

En esta sección deberán implementar el loop de entrenamiento de su red neuronal. Para esto, primero deben definir un `criterion`, en nuestro caso `nn.CrossEntropyLoss()` con la libreria de `pytorch`. Sucesivamente debera definir un optimizador, en nuestro caso `optim.SGD` desde el modulo `optim` de `pytorch`.

El loop de entrenamiento debe seguir la siguiente estructura:
```python
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:
    ...
```

donde `train_loader` proviene del iterador generado en la "sección iterador".

Dentro de "doble for" debera conjugar apropiadamente `opti.zero_grad()`, `loss = criterion(...)`, `loss.backward()` y `opti.step()` con tal de entrenar correctamente su red neuronal. Incluso entrenar, ya que a veces si no se hace de forma correcta entonces tristemente ¡su red no entrena!

> **Recomendación:** Puede guiarse del Auxiliar 2 para implementar el loop de entrenamiento.

### Preparación de la GPU y los datos de train/test

Importar la libreria `pytorch` y `numpy`

In [7]:
import torch
import numpy as np

Verificar que esta usando GPU. Sino, dirígase a **Runtime > Change runtime type** y seleccione la opción **T4 GPU**.

In [8]:
torch.cuda.is_available()

True

Preparación de los conjuntos train y test

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(tokenizer=lambda x: list(x), preprocessor=lambda x: x, token_pattern=None)

bow_train = pd.DataFrame(
    bow.fit_transform(train_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)
bow_test = pd.DataFrame(
    bow.transform(test_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)

bow_label_train = bow_train.astype(float).copy()
bow_label_test = bow_test.astype(float).copy()

map_from_class_to_int = {
    "?": 0,
    "+": 1,
    "-": 2
}

bow_label_train["class_"] = train_set["class_"]
bow_label_train["int_class_"] = train_set["class_"].apply(lambda x: map_from_class_to_int[x])

bow_label_test["class_"] = test_set["class_"]
bow_label_test["int_class_"] = test_set["class_"].apply(lambda x: map_from_class_to_int[x])

### Implementación (1.7 pts.)

#### Iterador de conjunto de datos
Implemente su clase `MyDataset` para acceder al dataset.

In [10]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, data, bow_cols):
        self.data = data
        self.bow_cols = bow_cols

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data.loc[index, "int_class_"])
        x_bow = torch.tensor(self.data.loc[index, self.bow_cols]. # Obtenemos el vector x_{index}
                             values.astype(float)).to(torch.float32) # y lo convertimos a tensor de float32
        return x_bow, label

Inicializar cada dataloader con sus cotenedor datos para train y test, y número de batches.

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    MyDataset(data = bow_label_train, bow_cols = bow_train.columns),
    batch_size = 5, num_workers = 1, shuffle=False)

test_loader = DataLoader(
    MyDataset(data = bow_label_test, bow_cols = bow_test.columns),
    batch_size = 5, num_workers = 1, shuffle=False)

Ejemplo de prueba para un batch de entrenamiento

In [12]:
batch = next(iter(train_loader))
print( batch )
print( batch[0].shape, batch[1].shape )

[tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

#### Modelo

Implemente a continuación su red neuronal

In [13]:
import torch.nn as nn
class MyNeuralNetwork(nn.Module):

    def __init__(self,
                 dim_vocab,
                 num_classes,
                 dim_hidden_input,
                 dim_hidden_output):
      """Inicializa la red neuronal

      Returns:
        None
      """

      super(MyNeuralNetwork, self).__init__()
      torch.manual_seed(42)

      # Primera capa
      self.first_layer = nn.Linear(dim_vocab, dim_hidden_input)

      # Capa oculta
      self.hidden_layer = nn.Linear(dim_hidden_input, dim_hidden_output)

      # Última capa
      self.last_layer = nn.Linear(dim_hidden_output, num_classes)

      # Función de activación
      self.relu = nn.ReLU(inplace=False)

    def forward(self, xs_bow):
      """Calcula la ultima capa mediante las capas intermedias de la red

      Args:
        xs_bow: Tensor

      Returns:
        Tensor con los valores de prediccion
      """

      ## Implementar aquí el forward-pass

      first_state = self.first_layer(xs_bow)
      first_state = self.relu(first_state)

      hidden_state = self.hidden_layer(first_state)
      hidden_state = self.relu(hidden_state)

      last_state = self.last_layer(hidden_state)

      return last_state

Ejemplo de prueba para su modelo NN para un batch de entrenamiento

In [14]:
test = MyNeuralNetwork(
    dim_vocab=len(train_loader.dataset.bow_cols),
    num_classes=3,
    dim_hidden_input=10,
    dim_hidden_output=5).cuda()

batch = next(iter(train_loader))

test(batch[0].cuda())

tensor([[-0.1008, -0.0804, -0.3952],
        [-0.1088, -0.0537, -0.4089],
        [-0.1051, -0.0697, -0.4043],
        [-0.0978, -0.0585, -0.4090],
        [-0.0992, -0.0623, -0.4076]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

#### Entrenamiento

Consideren las siguientes funciones que les serán utiles. Si lo desea puede modificarlas a su conveniencia.

In [15]:
def get_loss(net, iterator, criterion):
    net.eval()
    total_loss = 0
    num_evals = 0
    with torch.no_grad():
      for xs_bow, labels in iterator:
          xs_bow, labels = xs_bow.cuda(), labels.cuda()

          logits = net(xs_bow)

          loss = criterion(logits, labels)

          total_loss += loss.item() * xs_bow.shape[0]
          num_evals += xs_bow.shape[0]

    return total_loss / num_evals

def get_preds_tests_nn(net, iterator):
  net.eval()
  preds, tests = [], []
  with torch.no_grad():
    for xs_bow, labels in iterator:
      xs_bow, labels = xs_bow.cuda(), labels.cuda()

      logits = net(xs_bow)

      soft_probs = nn.Sigmoid()(logits)

      preds += np.argmax(soft_probs.tolist(), axis=1).tolist()
      tests += labels.tolist()

    return np.array(preds), np.array(tests)

A continuación, inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [16]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 5,
    "dim_hidden_output": 5,
    "learning_rate": 0.4,
    "epochs": 15
}


device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).cuda()

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().cuda()

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:

    opti.zero_grad()

    xs_bow, preds = xs_bow.to(device), labels.to(device)

    logits = net(xs_bow)

    loss = criterion(logits, preds)

    loss.backward()

    opti.step()

    #pass # Quitar esto cuando implementen el loop de entrenamiento

  total_loss = get_loss(net, train_loader, criterion)
  y_preds, y_tests = get_preds_tests_nn(net, train_loader)
  acc = (y_preds == y_tests).sum() / y_preds.shape[0]

  print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoca 0 completada! Loss: 1.1054631629410911 Accuracy: 0.38235294117647056
Epoca 1 completada! Loss: 1.0930101818898146 Accuracy: 0.38235294117647056
Epoca 2 completada! Loss: 1.0800085768980139 Accuracy: 0.38235294117647056
Epoca 3 completada! Loss: 1.056380967006964 Accuracy: 0.38235294117647056
Epoca 4 completada! Loss: 0.9195144391235184 Accuracy: 0.47058823529411764
Epoca 5 completada! Loss: 0.7975653784678263 Accuracy: 0.6176470588235294
Epoca 6 completada! Loss: 0.6394507672418567 Accuracy: 0.7058823529411765
Epoca 7 completada! Loss: 0.4972488123594838 Accuracy: 0.7647058823529411
Epoca 8 completada! Loss: 0.5996851416523842 Accuracy: 0.6176470588235294
Epoca 9 completada! Loss: 0.33634046876036067 Accuracy: 0.8235294117647058
Epoca 10 completada! Loss: 0.2385189199601026 Accuracy: 0.8823529411764706
Epoca 11 completada! Loss: 0.0932115251198411 Accuracy: 1.0
Epoca 12 completada! Loss: 0.05484988332233008 Accuracy: 1.0
Epoca 13 completada! Loss: 0.03735693112727912 Accuracy: 1.

Pruebe su modelo entrenado con la función `get_preds_tests_nn`.

In [19]:
# Ya no necesitara calcular gradientes para hacer inferencia
for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net, train_loader)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00         7

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando la función `get_preds_tests_nn`.

In [20]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader)
print(classification_report(y_tests, y_preds))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.83      0.91         6
           2       1.00      1.00      1.00         3

    accuracy                           0.92        13
   macro avg       0.93      0.94      0.93        13
weighted avg       0.94      0.92      0.92        13



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [21]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 2,
    "dim_hidden_output": 2,
    "learning_rate": 0.4,
    "epochs": 15
}


device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).cuda()

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().cuda()

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:

    opti.zero_grad()

    xs_bow, preds = xs_bow.to(device), labels.to(device)

    logits = net(xs_bow)

    loss = criterion(logits, preds)

    loss.backward()

    opti.step()

    #pass # Quitar esto cuando implementen el loop de entrenamiento

  total_loss = get_loss(net, train_loader, criterion)
  y_preds, y_tests = get_preds_tests_nn(net, train_loader)
  acc = (y_preds == y_tests).sum() / y_preds.shape[0]

  print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoca 0 completada! Loss: 1.1127501410596512 Accuracy: 0.38235294117647056
Epoca 1 completada! Loss: 1.1071627560783834 Accuracy: 0.38235294117647056
Epoca 2 completada! Loss: 1.106783810783835 Accuracy: 0.38235294117647056
Epoca 3 completada! Loss: 1.1072266084306381 Accuracy: 0.38235294117647056
Epoca 4 completada! Loss: 1.1076231265769285 Accuracy: 0.38235294117647056
Epoca 5 completada! Loss: 1.1078704595565796 Accuracy: 0.38235294117647056
Epoca 6 completada! Loss: 1.1080077220411861 Accuracy: 0.38235294117647056
Epoca 7 completada! Loss: 1.108079913784476 Accuracy: 0.38235294117647056
Epoca 8 completada! Loss: 1.108116880935781 Accuracy: 0.38235294117647056
Epoca 9 completada! Loss: 1.1081355792634628 Accuracy: 0.38235294117647056
Epoca 10 completada! Loss: 1.1081449214149923 Accuracy: 0.38235294117647056
Epoca 11 completada! Loss: 1.108149546034196 Accuracy: 0.38235294117647056
Epoca 12 completada! Loss: 1.1081517952329971 Accuracy: 0.38235294117647056
Epoca 13 completada! Loss:

In [22]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.46      1.00      0.63         6
           2       0.00      0.00      0.00         3

    accuracy                           0.46        13
   macro avg       0.15      0.33      0.21        13
weighted avg       0.21      0.46      0.29        13



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 2,
    "dim_hidden_output": 3,
    "learning_rate": 0.4,
    "epochs": 15
}


device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).cuda()

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().cuda()

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:

    opti.zero_grad()

    xs_bow, preds = xs_bow.to(device), labels.to(device)

    logits = net(xs_bow)

    loss = criterion(logits, preds)

    loss.backward()

    opti.step()

    #pass # Quitar esto cuando implementen el loop de entrenamiento

  total_loss = get_loss(net, train_loader, criterion)
  y_preds, y_tests = get_preds_tests_nn(net, train_loader)
  acc = (y_preds == y_tests).sum() / y_preds.shape[0]

  print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoca 0 completada! Loss: 1.1125045892070322 Accuracy: 0.38235294117647056
Epoca 1 completada! Loss: 1.0627017161425423 Accuracy: 0.38235294117647056
Epoca 2 completada! Loss: 0.9082242793896619 Accuracy: 0.6470588235294118
Epoca 3 completada! Loss: 0.5982328030992957 Accuracy: 0.7941176470588235
Epoca 4 completada! Loss: 0.5247351463664981 Accuracy: 0.7941176470588235
Epoca 5 completada! Loss: 0.39796790688791694 Accuracy: 0.7941176470588235
Epoca 6 completada! Loss: 0.38397766074494405 Accuracy: 0.7941176470588235
Epoca 7 completada! Loss: 0.45184319438960624 Accuracy: 0.7941176470588235
Epoca 8 completada! Loss: 0.3842281356542919 Accuracy: 0.7941176470588235
Epoca 9 completada! Loss: 0.44230760918820544 Accuracy: 0.7941176470588235
Epoca 10 completada! Loss: 0.6602159306502846 Accuracy: 0.7058823529411765
Epoca 11 completada! Loss: 0.2532780595556152 Accuracy: 0.7941176470588235
Epoca 12 completada! Loss: 0.1787401082754594 Accuracy: 0.9705882352941176
Epoca 13 completada! Loss: 0.

In [24]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         3

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



In [25]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 50,
    "dim_hidden_output": 50,
    "learning_rate": 0.4,
    "epochs": 15
}


device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).cuda()

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().cuda()

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:

    opti.zero_grad()

    xs_bow, preds = xs_bow.to(device), labels.to(device)

    logits = net(xs_bow)

    loss = criterion(logits, preds)

    loss.backward()

    opti.step()

    #pass # Quitar esto cuando implementen el loop de entrenamiento

  total_loss = get_loss(net, train_loader, criterion)
  y_preds, y_tests = get_preds_tests_nn(net, train_loader)
  acc = (y_preds == y_tests).sum() / y_preds.shape[0]

  print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoca 0 completada! Loss: 1.0768816400976742 Accuracy: 0.38235294117647056
Epoca 1 completada! Loss: 1.0434374888153637 Accuracy: 0.38235294117647056
Epoca 2 completada! Loss: 0.8320474607103011 Accuracy: 0.5588235294117647
Epoca 3 completada! Loss: 0.4608185668202007 Accuracy: 0.7941176470588235
Epoca 4 completada! Loss: 0.15261032925370863 Accuracy: 0.9705882352941176
Epoca 5 completada! Loss: 0.04222384873120224 Accuracy: 1.0
Epoca 6 completada! Loss: 0.01933412893456133 Accuracy: 1.0
Epoca 7 completada! Loss: 0.012454775901620878 Accuracy: 1.0
Epoca 8 completada! Loss: 0.008946328266414212 Accuracy: 1.0
Epoca 9 completada! Loss: 0.006895440984996693 Accuracy: 1.0
Epoca 10 completada! Loss: 0.005562900127295186 Accuracy: 1.0
Epoca 11 completada! Loss: 0.004639236535534591 Accuracy: 1.0
Epoca 12 completada! Loss: 0.003954481490997269 Accuracy: 1.0
Epoca 13 completada! Loss: 0.0034342460875289842 Accuracy: 1.0
Epoca 14 completada! Loss: 0.0030296789208317503 Accuracy: 1.0


In [26]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      0.83      0.91         6
           2       0.75      1.00      0.86         3

    accuracy                           0.92        13
   macro avg       0.92      0.94      0.92        13
weighted avg       0.94      0.92      0.93        13



Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de `(dim_hidden_input, dim_hidden_output)`.

**Comentarios:**

Se puede ver que el modelo requiere un minimo de neuronas para comenzar a aprender y disminuir la loss, como lo es el caso de la combinación (2,2) para `(dim_hidden_input, dim_hidden_output)`, se puede ver que no es suficiente para empezar a generalizar, sin embargo al aumentar a (2,3) se puede ver que se logra el mejor desempaño de todas las combinaciones, esto puede ser debido a que el problema es bastante simple por lo que las capas de 2 y 3 neuronas son capaces de generalizar con buena presicion, mientras que si se aumenta el numero de neuronas como lo es (5,5) y (50,50), si bien, tiene bastante buen rendimiento no supera al anteriormente mencionado, esto puede ser debido a que una gran cantidad de neuronas en todas esas epocas esten overfiteando el modelo sobre el entrenamiento, teniendo dificultades para aplicarlo en el conjunto de test.

En términos del modelo se puede ver que puede ver que en general el rendimiento es bueno si se aprende en cada epoca, los modelos en general tienen cierta facilidad para identificar la clase 0, esto se puede deber a que generalmente el simbolo '?' se encuentra en todas las oraciones de esta clase, despues el resto de clase tiene un rendimiento de clasificacion similar.
